In [271]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import math
import os
from shapely.geometry import Point

%matplotlib inline

pd.options.mode.chained_assignment = None


In [287]:
input_folder = "/Users/kazv/VanBovenDrive/VanBoven MT/500 Projects/P2 Wiertsema/data/Sonderingen/Sonderingen MOS/"
file_list = os.listdir(input_folder)
valid_files = [x for x in file_list if x.endswith('.gef')]


In [360]:
def gdf_from_gef(GEF):
    #Read lines
    with codecs.open(GEF, 'r', encoding='utf-8',errors='ignore') as fdata:
        contents = fdata.readlines()
    
    #Get info from GEF file metadata
    columnsep = [s for s in contents if "#COLUMNSEPARATOR" in s][0].split(' ')[1][0]
    XYID = [s for s in contents if "#XYID" in s][0].split(' ')
    x = float(XYID[2][:-1])
    y = float(XYID[3][:-1])

    EOH_line = [s for s in contents if "#EOH=" in s] #Last line before data starts
    start_line = contents.index(EOH_line[0])+1#line where data starts

    #Create dataframe from GEF file
    
    df = pd.DataFrame(x.split(columnsep) for x in contents[start_line::])
    #Drop last column, which only holds line seperators
    df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
    
    #Set column names
    col_info = [s for s in contents if "#COLUMNINFO= " in s]
    cols = []
    for info in col_info:
        info_split = info.split(',')
        unit = info_split[1].strip()
        quantity = info_split[2].strip()
        col_name = quantity+' [{}]'.format(unit)
        cols.append(col_name)
    df.columns = cols
    
    
    #Convert to GeoDataFrame and add geometry
    gdf = gpd.GeoDataFrame(df)
    gdf.crs = {'init': 'epsg:28992'}
    gdf['geometry']= Point(x,y)
    
    return gdf


In [378]:
gdf = gpd.GeoDataFrame([])
for file in valid_files:
    GEF = os.path.join(input_folder, file)
    gdf_new = gdf_from_gef(GEF)
    gdf = gdf.append(gdf_new, sort=False)
    


In [381]:
output_file = "/Users/kazv/VanBovenDrive/VanBoven MT/500 Projects/P2 Wiertsema/data/GEF_output.shp"
gdf.to_file(driver = 'ESRI Shapefile', filename = output_file)

/Users/kazv/miniconda3/envs/wiertsema/lib/python3.7/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [368]:
with codecs.open(GEF, 'r', encoding='utf-8',errors='ignore') as fdata:
    contents = fdata.readlines()

#Get info from GEF file metadata
columnsep = [s for s in contents if "#COLUMNSEPARATOR" in s][0].split(' ')[1][0]
XYID = [s for s in contents if "#XYID" in s][0].split(' ')
x = float(XYID[2][:-1])
y = float(XYID[3][:-1])
EOH_line = [s for s in contents if "#EOH=" in s]

In [376]:
start_line = contents.index(EOH_line[0])+1#line where data starts

#Create dataframe from GEF file
df = pd.DataFrame(x.split(columnsep) for x in contents[start_line::])

In [383]:
gdf['Sondeerlengte [m]'].to_numeric

AttributeError: 'Series' object has no attribute 'to_numeric'

In [386]:
gdf['Sondeerlengte [m]'] = pd.to_numeric(gdf['Sondeerlengte [m]'])

TypeError: arg must be a list, tuple, 1-d array, or Series